#  AI Game
## Name: Fatemeh Naeinian

### Student ID: 810099018


## Definition of project:
In this game, the first agent who makes a triangle with three similar color, loses the game. There is 6 points which could have 15 edges. Two agents will play this game. 

In [71]:
from select import select
from symbol import dotted_as_name
import turtle
import math
import random
from time import sleep
from sys import argv
import copy
from time import time

### minimax

There is two agents, blue acts randomly and red acts depend on minimax function. In minimax function, red chooses the movement with larger heuristic cost. It is assumed that blue chooses the movement with lower heuristic cost. So, for an specific depth, a tree of possible movements will be made. Then according to their heuristic costs, minimax function finds the best movement.

### heuristic

There is function called gameover() which helps us to predict whether an agent is going to win or not. Besides this function, it's better for an agent to last in greater depth. To act better, it's important to do the best. In a special state that both last in greater depth and none of them is going to lose, it's become more important that the agent choose the edge that has less chance to become a triangle.


In [72]:

class Sim:
    # Set true for graphical interface
    GUI = False
    screen = None
    selection = []
    turn = ''
    dots = []
    red = []
    blue = []
    available_moves = []
    minimax_depth = 0
    prune = False

    def __init__(self, minimax_depth, prune, gui):
        self.GUI = gui
        self.prune = prune
        self.minimax_depth = minimax_depth
        if self.GUI:
            self.setup_screen()

    def setup_screen(self):
        self.screen = turtle.Screen()
        self.screen.setup(800, 800)
        self.screen.title("Game of SIM")
        self.screen.setworldcoordinates(-1.5, -1.5, 1.5, 1.5)
        self.screen.tracer(0, 0)
        turtle.hideturtle()

    def draw_dot(self, x, y, color):
        turtle.up()
        turtle.goto(x, y)
        turtle.color(color)
        turtle.dot(15)

    def gen_dots(self):
        r = []
        for angle in range(0, 360, 60):
            r.append((math.cos(math.radians(angle)), math.sin(math.radians(angle))))
        return r

    def initialize(self):
        self.selection = []
        self.available_moves = []
        for i in range(0, 6):
            for j in range(i, 6):
                if i != j:
                    self.available_moves.append((i, j))
        if random.randint(0, 2) == 1:
            self.turn = 'red'
        else:
            self.turn = 'blue'
        self.dots = self.gen_dots()
        self.red = []
        self.blue = []
        if self.GUI: turtle.clear()
        self.draw()

    def draw_line(self, p1, p2, color):
        turtle.up()
        turtle.pensize(3)
        turtle.goto(p1)
        turtle.down()
        turtle.color(color)
        turtle.goto(p2)

    def draw_board(self):
        for i in range(len(self.dots)):
            if i in self.selection:
                self.draw_dot(self.dots[i][0], self.dots[i][1], self.turn)
            else:
                self.draw_dot(self.dots[i][0], self.dots[i][1], 'dark gray')

    def draw(self):
        if not self.GUI: return 0
        self.draw_board()
        for i in range(len(self.red)):
            self.draw_line((math.cos(math.radians(self.red[i][0] * 60)), math.sin(math.radians(self.red[i][0] * 60))),
                           (math.cos(math.radians(self.red[i][1] * 60)), math.sin(math.radians(self.red[i][1] * 60))),
                           'red')
        for i in range(len(self.blue)):
            self.draw_line((math.cos(math.radians(self.blue[i][0] * 60)), math.sin(math.radians(self.blue[i][0] * 60))),
                           (math.cos(math.radians(self.blue[i][1] * 60)), math.sin(math.radians(self.blue[i][1] * 60))),
                           'blue')
        self.screen.update()
        sleep(1)

    def _evaluate(self, red, blue, total_depth, curr_depth, player_turn, available_moves):  # heuristic
        # TODO
        if curr_depth == 0 or len(available_moves) == 0:
            return 0

        cost = (total_depth - curr_depth) * 17

        unique_edge = 10
        repeted_edge = 15
        for i in range(6):
            count = 0
            repeted_point = []
            for j in range(len(red)):
                if red[j][0] == i:
                    count += 1
                if red[j][1] == i:
                    repeted_point.append(j)
                    count += 1
            if  count == 1:
                cost += unique_edge
            if count > 1:
                cost -= count * repeted_edge

        for i in range(6):
            count = 0
            for j in range(len(blue)):
                if blue[j][0] == i:
                    count += 1
                if blue[j][1] == i:
                    repeted_point.append(j)
                    count += 1
            if count == 1:
                cost -= unique_edge
            if count > 1:
                cost += count * repeted_edge

        who_wins_with_this_move = self.gameover(red, blue)
        if who_wins_with_this_move != 0:
            if who_wins_with_this_move == "blue":
                cost += -100
            elif who_wins_with_this_move == "red":
                cost += 100
            return cost

#         if total_depth - curr_depth > 1 :
#             return cost

        if player_turn == 'red':  # maximize
            cost_value = -math.inf
            for move in available_moves:
                red.append(move)
                available_moves.remove(move)
                new_cost = self._evaluate(red, blue, total_depth, curr_depth-1, 'blue', available_moves)
                red.remove(move)
                available_moves.append(move)
                if new_cost > cost_value:
                    cost_value = new_cost
            return cost_value

        else:  # minimize
            cost_value = math.inf
            for move in available_moves:
                blue.append(move)
                available_moves.remove(move)
                new_cost = self._evaluate(red, blue, total_depth, curr_depth-1, 'red', available_moves)
                blue.remove(move)
                available_moves.append(move)
                if new_cost < cost_value:
                    cost_value = new_cost
            return cost_value



    def minimax(self, depth, player_turn):
        # TODO

        if player_turn == 'red':  # maximize
            cost_value = -math.inf
            available_moves = self.available_moves
            selected_move = 0
            red = copy.deepcopy(self.red)
            blue = copy.deepcopy(self.blue)
            for move in available_moves:
                red.append(move)
                available_moves.remove(move)
                new_cost = self._evaluate(red, blue, depth, depth-1, 'blue', available_moves)
                red.remove(move)
                available_moves.append(move)
                if new_cost > cost_value:
                    cost_value = new_cost
                    selected_move = move
            return selected_move, cost_value

        else:  # minimize
            cost_value = math.inf
            available_moves = self.available_moves
            selected_move = 0
            red = copy.deepcopy(self.red)
            blue = copy.deepcopy(self.blue)
            for move in available_moves:
                blue.append(move)
                available_moves.remove(move)
                new_cost = self._evaluate(red, blue, depth-1, 'blue', available_moves)
                blue.remove(move)
                available_moves.append(move)
                if new_cost < cost_value:
                    cost_value = new_cost
                    selected_move = move
            return selected_move, cost_value

    def enemy(self):
        return random.choice(self.available_moves)

    def _swap_turn(self, current_turn):
        if current_turn == 'blue':
            return 'red'
        else:
            return 'blue'

    def play(self):
        self.initialize()
        while True:
            if self.turn == 'red':
                selection = self.minimax(depth=self.minimax_depth, player_turn=self.turn)[0]
                # selection = self.enemy()
                if selection[1] < selection[0]:
                    selection = (selection[1], selection[0])
            else:
                selection = self.enemy()
                if selection[1] < selection[0]:
                    selection = (selection[1], selection[0])
            if selection in self.red or selection in self.blue:
                raise Exception("Duplicate Move!!!")
            if self.turn == 'red':
                self.red.append(selection)
            else:
                self.blue.append(selection)

            self.available_moves.remove(selection)
            self.turn = self._swap_turn(self.turn)
            selection = []
            self.draw()
            r = self.gameover(self.red, self.blue)
            if r != 0:
                return r

    def gameover(self, r, b):
        if len(r) < 3:
            return 0
        r.sort()
        for i in range(len(r) - 2):
            for j in range(i + 1, len(r) - 1):
                for k in range(j + 1, len(r)):
                    if r[i][0] == r[j][0] and r[i][1] == r[k][0] and r[j][1] == r[k][1]:
                        return 'blue'
        if len(b) < 3: return 0
        b.sort()
        for i in range(len(b) - 2):
            for j in range(i + 1, len(b) - 1):
                for k in range(j + 1, len(b)):
                    if b[i][0] == b[j][0] and b[i][1] == b[k][0] and b[j][1] == b[k][1]:
                        return 'red'
        return 0




In [73]:
if __name__ == "__main__":
    minimax_depth = [1, 3, 5]
    for i in range(len(minimax_depth)):
        t1 = time()
        game = Sim(minimax_depth = minimax_depth[i], prune=True, gui=0)
        results = {'red': 0, 'blue': 0}
        for j in range(200):
            results[game.play()] += 1
        t2 = time()
        print('depth = ',minimax_depth[i])
        print('WIN RESULTS = ',results)
        print('chance of wining = ', results['red']/200)
        print('Average time for each game with depth '+ str(minimax_depth[i]) + ' = ' + str((t2-t1)/200), ' seconds\n')

depth =  1
WIN RESULTS =  {'red': 82, 'blue': 118}
chance of wining =  0.41
Average time for each game with depth 1 = 0.00041928648948669434  seconds

depth =  3
WIN RESULTS =  {'red': 166, 'blue': 34}
chance of wining =  0.83
Average time for each game with depth 3 = 0.020010079145431518  seconds

depth =  5
WIN RESULTS =  {'red': 178, 'blue': 22}
chance of wining =  0.89
Average time for each game with depth 5 = 1.9665961587429046  seconds



Let's check the results. With depth=1, it's hard to predict the game, I think the game will be more like acting randomly.
With depth=3 results gets better, and with depth=5 chance of wining reaches to 89% that is really good. It's because of minimax function which helps to predict the game.

# Alpha-Beta Pruning
Two variables called Alpha and Beta are cosidered to prune the tree of possible movements. With pruning, algorithm gets faster because whenever a better solution is found, the other branches of the tree won't be checked.

In [74]:
from select import select
from symbol import dotted_as_name
import turtle
import math
import random
from time import sleep
from sys import argv
import copy


class Sim:
    # Set true for graphical interface
    GUI = False
    screen = None
    selection = []
    turn = ''
    dots = []
    red = []
    blue = []
    available_moves = []
    minimax_depth = 0
    prune = False

    def __init__(self, minimax_depth, prune, gui):
        self.GUI = gui
        self.prune = prune
        self.minimax_depth = minimax_depth
        if self.GUI:
            self.setup_screen()

    def setup_screen(self):
        self.screen = turtle.Screen()
        self.screen.setup(800, 800)
        self.screen.title("Game of SIM")
        self.screen.setworldcoordinates(-1.5, -1.5, 1.5, 1.5)
        self.screen.tracer(0, 0)
        turtle.hideturtle()

    def draw_dot(self, x, y, color):
        turtle.up()
        turtle.goto(x, y)
        turtle.color(color)
        turtle.dot(15)

    def gen_dots(self):
        r = []
        for angle in range(0, 360, 60):
            r.append((math.cos(math.radians(angle)), math.sin(math.radians(angle))))
        return r

    def initialize(self):
        self.selection = []
        self.available_moves = []
        for i in range(0, 6):
            for j in range(i, 6):
                if i != j:
                    self.available_moves.append((i, j))
        if random.randint(0, 2) == 1:
            self.turn = 'red'
        else:
            self.turn = 'blue'
        self.dots = self.gen_dots()
        self.red = []
        self.blue = []
        if self.GUI: turtle.clear()
        self.draw()

    def draw_line(self, p1, p2, color):
        turtle.up()
        turtle.pensize(3)
        turtle.goto(p1)
        turtle.down()
        turtle.color(color)
        turtle.goto(p2)

    def draw_board(self):
        for i in range(len(self.dots)):
            if i in self.selection:
                self.draw_dot(self.dots[i][0], self.dots[i][1], self.turn)
            else:
                self.draw_dot(self.dots[i][0], self.dots[i][1], 'dark gray')

    def draw(self):
        if not self.GUI: return 0
        self.draw_board()
        for i in range(len(self.red)):
            self.draw_line((math.cos(math.radians(self.red[i][0] * 60)), math.sin(math.radians(self.red[i][0] * 60))),
                           (math.cos(math.radians(self.red[i][1] * 60)), math.sin(math.radians(self.red[i][1] * 60))),
                           'red')
        for i in range(len(self.blue)):
            self.draw_line((math.cos(math.radians(self.blue[i][0] * 60)), math.sin(math.radians(self.blue[i][0] * 60))),
                           (math.cos(math.radians(self.blue[i][1] * 60)), math.sin(math.radians(self.blue[i][1] * 60))),
                           'blue')
        self.screen.update()
        sleep(1)

    def _evaluate(self, red, blue, total_depth, curr_depth, player_turn, available_moves, alpha, beta):  # heuristic
        # TODO
        if curr_depth == 0 or len(available_moves) == 0:
            return 0

        cost = (total_depth - curr_depth) * 17

        unique_edge = 10
        repeted_edge = 15
        for i in range(6):
            count = 0
            repeted_point = []
            for j in range(len(red)):
                if red[j][0] == i:
                    count += 1
                if red[j][1] == i:
                    repeted_point.append(j)
                    count += 1
            if  count == 1:
                cost += unique_edge
            if count > 1:
                cost -= count * repeted_edge

        for i in range(6):
            count = 0
            for j in range(len(blue)):
                if blue[j][0] == i:
                    count += 1
                if blue[j][1] == i:
                    repeted_point.append(j)
                    count += 1
            if count == 1:
                cost -= unique_edge
            if count > 1:
                cost += count * repeted_edge

        who_wins_with_this_move = self.gameover(red, blue)
        if who_wins_with_this_move != 0:
            if who_wins_with_this_move == "blue":
                cost += -100
            elif who_wins_with_this_move == "red":
                cost += 100
            return cost

#         if total_depth - curr_depth > 1 :
#             return cost

        if player_turn == 'red':  # maximize
            cost_value = -math.inf
            for move in available_moves:
                red.append(move)
                available_moves.remove(move)
                new_cost = self._evaluate(red, blue, total_depth, curr_depth-1, 'blue', available_moves, alpha, beta)
                # print(new_cost)
                red.remove(move)
                available_moves.append(move)
                if new_cost > cost_value:
                    cost_value = new_cost
                alpha = max(alpha, cost_value)
                if alpha >= beta:
                    return cost_value
            return cost_value

        else:  # minimize
            cost_value = math.inf
            for move in available_moves:
                blue.append(move)
                available_moves.remove(move)
                new_cost = self._evaluate(red, blue, total_depth, curr_depth-1, 'red', available_moves, alpha, beta)
                # print(new_cost)
                blue.remove(move)
                available_moves.append(move)
                if new_cost < cost_value:
                    cost_value = new_cost
                beta = min(beta, cost_value)
                if alpha >= beta:
                    return cost_value
            return cost_value



    def minimax(self, depth, player_turn, alpha, beta):
        # TODO
        if player_turn == 'red':  # maximize
            cost_value = -math.inf
            available_moves = self.available_moves
            random.shuffle(available_moves)
            selected_move = 0
            red = copy.deepcopy(self.red)
            blue = copy.deepcopy(self.blue)
            for move in available_moves:
                red.append(move)
                available_moves.remove(move)
                new_cost = self._evaluate(red, blue, depth, depth-1, 'blue', available_moves, alpha, beta)
                red.remove(move)
                available_moves.append(move)
                if new_cost > cost_value:
                    cost_value = new_cost
                    selected_move = move
                alpha = max(alpha, cost_value)
                if cost_value >= beta:
                    return selected_move, cost_value
            return selected_move, cost_value

        else:  # minimize
            cost_value = math.inf
            available_moves = self.available_moves
            selected_move = 0
            red = copy.deepcopy(self.red)
            blue = copy.deepcopy(self.blue)
            for move in available_moves:
                blue.append(move)
                available_moves.remove(move)
                new_cost = self._evaluate(red, blue, depth-1, 'blue', available_moves)
                blue.remove(move)
                available_moves.append(move)
                if new_cost < cost_value:
                    cost_value = new_cost
                    selected_move = move
                beta = min(beta, cost_value)
                if alpha >= cost_value:
                    return selected_move, cost_value
            return selected_move, cost_value

    def enemy(self):
        return random.choice(self.available_moves)

    def _swap_turn(self, current_turn):
        if current_turn == 'blue':
            return 'red'
        else:
            return 'blue'

    def play(self):
        self.initialize()
        while True:
            if self.turn == 'red':
                selection = self.minimax(depth=self.minimax_depth, player_turn=self.turn, alpha=-math.inf, beta=math.inf)[0]
                # selection = self.enemy()
                if selection[1] < selection[0]:
                    selection = (selection[1], selection[0])
            else:
                selection = self.enemy()
                if selection[1] < selection[0]:
                    selection = (selection[1], selection[0])
            if selection in self.red or selection in self.blue:
                raise Exception("Duplicate Move!!!")
            if self.turn == 'red':
                self.red.append(selection)
            else:
                self.blue.append(selection)

            self.available_moves.remove(selection)
            self.turn = self._swap_turn(self.turn)
            selection = []
            self.draw()
            r = self.gameover(self.red, self.blue)
            if r != 0:
                return r

    def gameover(self, r, b):
        if len(r) < 3:
            return 0
        r.sort()
        for i in range(len(r) - 2):
            for j in range(i + 1, len(r) - 1):
                for k in range(j + 1, len(r)):
                    if r[i][0] == r[j][0] and r[i][1] == r[k][0] and r[j][1] == r[k][1]:
                        return 'blue'
        if len(b) < 3: return 0
        b.sort()
        for i in range(len(b) - 2):
            for j in range(i + 1, len(b) - 1):
                for k in range(j + 1, len(b)):
                    if b[i][0] == b[j][0] and b[i][1] == b[k][0] and b[j][1] == b[k][1]:
                        return 'red'
        return 0



In [75]:
if __name__ == "__main__":
    minimax_depth = [1, 3, 5, 7]
    for i in range(len(minimax_depth)):
        t1 = time()
        game = Sim(minimax_depth = minimax_depth[i], prune=True, gui=0)
        results = {'red': 0, 'blue': 0}
        for j in range(200):
            results[game.play()] += 1
        t2 = time()
        print('depth = ',minimax_depth[i])
        print('WIN RESULTS = ',results)
        print('chance of wining = ', results['red']/200)
        print('Average time for each game with depth '+ str(minimax_depth[i]) + ' = ' + str((t2-t1)/200), ' seconds\n')

depth =  1
WIN RESULTS =  {'red': 105, 'blue': 95}
chance of wining =  0.525
Average time for each game with depth 1 = 0.00046365737915039063  seconds

depth =  3
WIN RESULTS =  {'red': 177, 'blue': 23}
chance of wining =  0.885
Average time for each game with depth 3 = 0.0053498268127441405  seconds

depth =  5
WIN RESULTS =  {'red': 173, 'blue': 27}
chance of wining =  0.865
Average time for each game with depth 5 = 0.054407527446746824  seconds

depth =  7
WIN RESULTS =  {'red': 177, 'blue': 23}
chance of wining =  0.885
Average time for each game with depth 7 = 0.559383442401886  seconds



As we see, with pruning execution time decreases. This means alogorithm works faster and computational time and memory complexity decreases.

# Questions
1) A good heuristic helps agent to win. Heuristics are not guaranteed to succeed but often lead to problem solution. The heuristic used in this code considers almost important things, for example with using function gameover(), cost increases 100 points if red wins, if blue wins it decreases 100 points. If red choose an edge which is connected to other edges in one vetexes, cost decreases 15 points for each edge and if edge is unique and is not connected to others, cost increases 10 points. If blue does that, cost increases 15 points for each edge and if edge is unique and is not connected to others, cost decreases 10 points. 
the other important thing is that game is better to be longer, because it increases the chance of wining if blue agent makes a bad movement. So, cost increases 17 points in each depth.

2) When the depth gets higher, the chance of wining increases. In alogorithm without pruning results are: depth 1: 41% , depth 3: 83% and depth 5: 89% .

    When the depth gets higher, computational time increases. In alogorithm without pruning results are: depth 1: 0.0004s , depth 3: 0.02s and depth 5: 1.96s .

    Number of nodes increases by a factorial rate, for example if 10 nodes are still uncoloured for depth=5, tree has 10 * 9 * 8 * 7 * 6 branches.


3) Available moves are added by numerical order, but before starting to search with minimax, Available moves would be shuffled. because an specific pattern of adding nodes is not going to help. So with shuffling, chance of having bias descreases.